In [1]:
import csv
import subprocess
import json
import requests
import lizard
from pydriller import RepositoryMining

# create the project URL list
url_list = []
with open('topJavaMavenProjects.csv') as csv_file:
    i = 0
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if i == 0:
            i += 1
            continue
        url_list.append(row[0])

#  create the base table (cm_pj_usr) where
# key: commit_id
# value: project_info (dic), user_info (dic), commit_info (dic),bug_num
f = open("sstubs.json")
sstubs = json.load(f)

# get the commit_id set
# commit_id_set = set()
# for item in sstubs:
#     commit_id_set.add(item["fixCommitSHA1"])

# cm_pj_usr = {}

'''
method 1, I think it may not work
for item in sstubs:
    
    project_name = item["projectName"].split(".")[1]
    commit_id = item["fixCommitSHA1"]
    user_name = "xinkai li" # cannot get it need use pydriller
    #project_url = project_dic[project_name]
    
    #for commit in RepositoryMining(project_url, only_commits=[commit_id]).traverse_commits():
    #    user_name = commit.author.name
    
    pj_usr_name = project_name + " " + user_name
    bug_num = 1
    if commit_id in cm_pj_usr.keys():
        bug_num = cm_pj_usr[commit_id]["bug_num"]
        bug_num += 1
    cm_pj_usr[commit_id] = {
        "project_info": project_name,
        "user_info": pj_usr_name,
        "bug_num": bug_num
        }
'''
# for commit in RepositoryMining("../spring-boot-master").traverse_commits():
#     commit_id = str(commit.hash)
#     if commit_id in commit_id_set: # this is a simple stupid bug
#         project_name = commit.project_name
#         user_name = commit.author.name
#         pj_usr_name = project_name + " " + user_name # avoid repeat
#         bug_num = 1
#         commit_info_dic = {}

#         if commit_id in cm_pj_usr.keys():
#             bug_num = cm_pj_usr[commit_id]["bug_num"]
#             bug_num += 1
#         cm_pj_usr[commit_id] = {
#             "project_info": project_name,
#             "user_info": pj_usr_name,
#             "commit_info": commit_info_dic,
#             "bug_num": bug_num
#             }

# print(cm_pj_usr)

bug_inducing_commits = {}
for item in sstubs:
    commitId = item["fixCommitParentSHA1"]
    owner = item['projectName'].split(".")[0]
    repo = item['projectName'].split(".")[1]
    if commitId in bug_inducing_commits:
        bug_inducing_commits[commitId]['count'] += 1
    else:
        bug_inducing_commits[commitId] = {
            'count': 1,
            'owner': owner,
            'repo': repo
        } 

token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
headers = {'Authorization': "Token " + token}
rate_limit_url = "https://api.github.com/rate_limit"
req = requests.get(rate_limit_url, headers=headers).json()
print(req)

url_list = url_list[0:10]
print(url_list)

projects = {}
for url in url_list:
    items = url.split("/")
    owner = items[len(items)-2]
    repo = items[len(items)-1]
    project_api = 'https://api.github.com/repos/' + owner + '/' + repo
    token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
    headers = {'Authorization': "Token " + token}
    project = requests.get(project_api, headers=headers).json()
    created_at = project['created_at']
    updated_at = project['updated_at']
    size = project['size']
    stargazers_count = project['stargazers_count']
    watchers_count = project['watchers_count']
    forks_count = project['forks_count']
    network_count = project['network_count']
    subscribers_count = project['subscribers_count']
    project_info = {
        'Owner' : owner,
        'Name' : repo,
        'Created at' : created_at,
        'Updated at' : updated_at,
        'Size' : size,
        'Stargazers' : stargazers_count,
        'Watchers' : watchers_count,
        'Forks' : forks_count,
        'Network' : network_count,
        'Subscribers' : subscribers_count
    }
    key = owner + '.' + repo
    print(project_info)
    projects[key] = project_info
print(projects)


sliced_commits = {}
commit_keys = list(bug_inducing_commits)
for i in range(10):
    key = commit_keys[i]
    sliced_commits[key] = bug_inducing_commits[key]
print(sliced_commits)

commits = {}
for key, value in sliced_commits.items(): 
    owner = value['owner']
    repo = value['repo']
    commit_api = 'https://api.github.com/repos/' + owner + '/' + repo + '/commits/' + key
    token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
    headers = {'Authorization': "Token " + token}
    commit = requests.get(commit_api, headers=headers).json()
    author_name = commit['commit']['author']['name']
    author_email = commit['commit']['author']['email']
    loc = commit['stats']['total']
    additions = commit['stats']['additions']
    deletions = commit['stats']['deletions']
    files_in_change = len(commit['files'])
    commit_info = {
        'Author name': author_name,
        'Author email': author_email,
        'LOC': loc,
        'Add': additions,
        'Delete': deletions,
        'Files': files_in_change
    }
    print(commit_info)
    commits[key] = commit_info
print(commits)


# 'e3c87cba0bbed00c739e01599e29a5621c9ef3b3': 
# {'project_info': 'spring-boot', 
# 'user_info': 'spring-boot Andy Wilkinson', 
# 'commit_info': {}, 
# 'bug_num': 1}
# }

# build the project_info table where
# key: project_name
# value: project_info

# build the user_info table where
# key: project_user_names
# value: user_info

# go through the cm_pj_usr table
# use project_name to get project_info
# use pj_usr_name to get user_info
# save data in format (bug_num, project_info, user_info) into a csv



{'resources': {'core': {'limit': 5000, 'used': 354, 'remaining': 4646, 'reset': 1617187586}, 'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1617186606}, 'graphql': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1617190146}, 'integration_manifest': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1617190146}, 'source_import': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1617186606}, 'code_scanning_upload': {'limit': 500, 'used': 0, 'remaining': 500, 'reset': 1617190146}}, 'rate': {'limit': 5000, 'used': 354, 'remaining': 4646, 'reset': 1617187586}}
['https://github.com/spring-projects/spring-boot', 'https://github.com/iluwatar/java-design-patterns', 'https://github.com/square/retrofit', 'https://github.com/square/okhttp', 'https://github.com/zxing/zxing', 'https://github.com/libgdx/libgdx', 'https://github.com/google/guava', 'https://github.com/alibaba/dubbo', 'https://github.com/jfeinstein10/SlidingMenu', 'https://github.com/netty/netty']
{'Owner': '